In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 193 kB/s 


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor

import warnings
warnings.filterwarnings(action="ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#------------------------------------------------------------------ 학습데이터셋
from sklearn.model_selection import train_test_split

#------------------------------------------------------------------ CART(Classification and Regression Tree)
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVC, NuSVR
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor,  AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet  #----------------*****
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix


#------------------------------------------------------------------ 증강학습
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV
from imblearn.over_sampling import SMOTE

#------------------------------------------------------------------ 스케일링
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


#------------------------------------------------------------------ 평가관련
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
# MAE
# MSE  : (squared=True)
# RMSE : (squared=False)
# MSLE


# 모델
from catboost import CatBoostClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier,RandomForestRegressor
from lightgbm import LGBMClassifier
from tqdm import tqdm


#------------------------------------------------------------------ 튜닝관련

import warnings
warnings.filterwarnings(action='ignore')
sns.set()

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/real_final_data_for_modeling.csv')

# 이후에 gan쓸때 전역변수 선언을 위해 이름 변경
df.rename(columns={' 당기순이익(손실)':'당기순이익'}, inplace= True)
df.rename(columns={'유보액/납입자본비율':'유보액_납입자본비율'}, inplace =True)


# selected feature df 불러오기
feat_df = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/feature_selection_df.csv')


# selected_x_data column 반환
def return_selected_data(df_hj, model, feature_list_name):
  df=df_hj.copy()
  for i in df.columns[1:]:
    df[i] = df[i].apply(lambda x : x[2:-2].replace("'",'').split(', '))
  tmp = df[df['model']==model][feature_list_name].iloc[0]




  return tmp
  


In [ ]:
return_selected_data(feat_df,"RF",'rfe_features_list')

['총자산증가율',
 '매출액증가율',
 '총자본영업이익율',
 '총자본순이익율',
 '매출액영업이익율',
 '자기자본비율',
 '부채비율',
 '유보액/납입자본비율',
 '총자본회전율',
 '자기자본회전율',
 ' 당기순이익(손실)',
 '영업활동으로인한현금흐름',
 'Z_score',
 '부도확률_O_score',
 'K_score',
 '자본잠식',
 '투자주의환기종목',
 'PER',
 'PBR',
 'EPS',
 'EPS성장률']

# Build Model

In [ ]:
# 학습한 모델 반환
def build_model(model, X, y):
  clf = model
  clf.fit(X, y)
  return clf


# 모델 검증
def evaluate_model(clf, X, y):


  pred = clf.predict(X)
  
  
  score_list = []
  Accuracy = round(accuracy_score(y, pred), 4)
  Precision = round(recall_score(y, pred, average='macro'), 4)
  F1 = round(f1_score(y, pred, average='macro'), 4)
  Recall = round(recall_score(y, pred, average='macro'),4)

  score_list.append(Accuracy)
  score_list.append(Precision)
  score_list.append(F1)
  score_list.append(Recall)
  model_eval = pd.DataFrame([score_list], columns = ['Accuracy', 'Precision', 'F1', 'Recall'])
  
  cf_matrix = confusion_matrix(y, pred)

  return model_eval, cf_matrix



# 모델 성능 저장
def model_eval_data(clf, X_train, y_train,
                    X_test, y_test,
                    model_eval_train,
                    model_eval_test,
                    Name = None):
  temp_eval_train, cf_matrix_train = evaluate_model(clf, X_train, y_train)
  temp_eval_test, cf_matrix_test = evaluate_model(clf, X_test, y_test)
  temp_eval_train.index = [Name]
  temp_eval_test.index = [Name]

  try:
    model_eval_train = model_eval_train.append(temp_eval_train)
    model_eval_test = model_eval_test.append(temp_eval_test)
  except:
    model_eval_train = temp_eval_train
    model_eval_test = temp_eval_test

  return model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test



# 초기화
# model_eval_train = pd.DataFrame({}, [])
# model_eval_test = pd.DataFrame({}, [])
model_eval_train = pd.DataFrame()
model_eval_test = pd.DataFrame()

# 1. SMOTE

In [ ]:
# feature selection 이후 파트 다시 정리하기

y_data = df['관리종목여부']
x_data = df.drop(["관리종목여부"], axis=1)

# data augmentation 이전에 미리 원본 데이터로 테스트 데이터셋 구축(stratify 기능을 통해 균형 있게 테스트셋 구축)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1123, stratify=y_data) # 데이터를 균등하게 나누고 test 검증하기 위해

print("shape 확인")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print()
print(f'train 데이터 타겟값 분포 \n{pd.Series(y_train).value_counts()}')
print()
print(f'test 데이터 타겟값 분포 \n{pd.Series(y_test).value_counts()}')

shape 확인
(4092, 45)
(4092,)
(1024, 45)
(1024,)

train 데이터 타겟값 분포 
0.0    3999
1.0      93
Name: 관리종목여부, dtype: int64

test 데이터 타겟값 분포 
0.0    1001
1.0      23
Name: 관리종목여부, dtype: int64


In [ ]:
# feature selection 이후 파트 다시 정리하기

y_data = df['관리종목여부']
x_data = df.drop(["관리종목여부"], axis=1)

# data augmentation 이전에 미리 원본 데이터로 테스트 데이터셋 구축(stratify 기능을 통해 균형 있게 테스트셋 구축)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1123, stratify=y_data) # 데이터를 균등하게 나누고 test 검증하기 위해



# smote 적용
def smote_eval(model, x_train, x_test, y_train, y_test, select_method):
  from imblearn.over_sampling import SMOTE
  smote = SMOTE(random_state=1123)
  X_train_over, y_train_over = smote.fit_resample(x_train, y_train)
  print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트:', x_train.shape, y_train.shape)
  print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트:', X_train_over.shape, y_train_over.shape)
  print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())
  
  clf = build_model(model, X_train_over, y_train_over) # 모델 학습


  # pred = model.predict(x_test)
  # pred_proba = model.predict_proba(x_test)
  # # cross_val_score(model, X_train_sm, y_train_sm, cv=skf, scoring='roc_auc').mean()


  # print('Confusion matrix')
  # print(confusion_matrix(y_test, pred))


  # print(f"Precision : {precision_score(y_test, pred, average='macro'):.4f}")
  # print(f"Recall    : {recall_score(y_test, pred, average='macro'):.4f}")
  # print(f"F1        : {f1_score(y_test, pred, average='macro'):.4f}")
  # print(f"Accuracy  : {accuracy_score(y_test, pred):.4f}")

  return model_eval_data(clf, X_train_over, y_train_over, x_test, y_test,  model_eval_train,
                                                                                     model_eval_test, Name = f'{model} & SMOTE method & {select_method}')
                                                                                   
                                                                                    
                                                                                    


In [ ]:
# model = CatBoostClassifier()
# model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test = smote_eval(model, x_train, x_test, y_train, y_test)
# model_eval_train

In [ ]:
# # 기본 모델(cat) && 모든 컬럼 && smote
# model_eval_test

## SMOTE - 모든 모델 & feature selection 성능평가

In [ ]:
# feature selection 이후 파트 다시 정리하기







for select_method in feat_df.columns[1:]:

  
  cat_clf = CatBoostClassifier(random_state = 1123)
  lda_clf = LDA(n_components=1)
  RF_clf = RandomForestClassifier(random_state=1123)
  EX_clf = ExtraTreesClassifier(random_state=1123)
  LGB_clf = LGBMClassifier(random_state=1123)
  GB_clf = GradientBoostingClassifier(random_state=1123)

  models = [ ("CF", cat_clf),
            ("LDA", lda_clf),
            ("RF", RF_clf),
            ("EX", EX_clf),
            ("LGB", LGB_clf),
            ('GB', GB_clf)
          ]
  for model_name, model in models:

    # 데이터 초기화
    y_data = df['관리종목여부']
    x_data = df.drop(["관리종목여부"], axis=1)


    x_data_col = return_selected_data(feat_df, model_name, select_method) # 모델별 feature select 방식별 선정된 feature 반환
    # data augmentation 이전에 미리 원본 데이터로 테스트 데이터셋 구축(stratify 기능을 통해 균형 있게 테스트셋 구축)
    #' 당기순이익(손실)' 처리
    if ' 당기순이익(손실)' in x_data_col:
      idx3 = x_data_col.index(' 당기순이익(손실)')
      x_data_col[idx3] = '당기순이익'

    if '유보액/납입자본비율' in x_data_col:
      idx4 = x_data_col.index('유보액/납입자본비율')
      x_data_col[idx4] = '유보액_납입자본비율'

    
    # feautre 선별
    new_x_data = x_data[x_data_col]
    new_y_data = y_data

    # 날짜 컬럼 빼주기
    if '날짜' in new_x_data.columns:
      new_x_data = new_x_data.drop('날짜', axis=1)


    # stratify를 통해 test 검증셋과 증강할 데이터셋 구분
    x_train, x_test, y_train, y_test = train_test_split(new_x_data, new_y_data, test_size=0.2, random_state=1123, stratify=y_data) # 데이터를 균등하게 나누고 test 검증하기 위해
    
    model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test = smote_eval(model, x_train, x_test, y_train, y_test, select_method)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
178:	learn: 0.0484848	total: 1.25s	remaining: 5.73s
179:	learn: 0.0482930	total: 1.25s	remaining: 5.72s
180:	learn: 0.0480684	total: 1.26s	remaining: 5.71s
181:	learn: 0.0478490	total: 1.27s	remaining: 5.69s
182:	learn: 0.0476310	total: 1.27s	remaining: 5.68s
183:	learn: 0.0473218	total: 1.28s	remaining: 5.67s
184:	learn: 0.0470611	total: 1.28s	remaining: 5.65s
185:	learn: 0.0468259	total: 1.29s	remaining: 5.64s
186:	learn: 0.0464479	total: 1.29s	remaining: 5.63s
187:	learn: 0.0462476	total: 1.3s	remaining: 5.61s
188:	learn: 0.0459670	total: 1.3s	remaining: 5.6s
189:	learn: 0.0454926	total: 1.31s	remaining: 5.59s
190:	learn: 0.0453526	total: 1.31s	remaining: 5.57s
191:	learn: 0.0450150	total: 1.32s	remaining: 5.56s
192:	learn: 0.0447652	total: 1.33s	remaining: 5.55s
193:	learn: 0.0446081	total: 1.33s	remaining: 5.53s
194:	learn: 0.0443805	total: 1.34s	remaining: 5.52s
195:	learn: 0.0442017	total: 1.34s	remaining: 5.51s
196:	learn: 0.0440507	total: 1.

In [ ]:
pd.DataFrame(model_eval_test['F1'].sort_values(ascending=False))

,F1
GradientBoostingClassifier(random_state=1123) & SMOTE method & select_k_feature_list,0.7213
RandomForestClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.7213
<catboost.core.CatBoostClassifier object at 0x7fb6cd87b610> & SMOTE method & perm_features_list,0.7138
GradientBoostingClassifier(random_state=1123) & SMOTE method & perm_features_list,0.7040
ExtraTreesClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.7031
<catboost.core.CatBoostClassifier object at 0x7fb6a1af9250> & SMOTE method & rfe_features_list,0.7013
RandomForestClassifier(random_state=1123) & SMOTE method & stepwise_feature_list,0.6840
LGBMClassifier(random_state=1123) & SMOTE method & stepwise_feature_list,0.6835
GradientBoostingClassifier(random_state=1123) & SMOTE method & stepwise_feature_list,0.6804
<catboost.core.CatBoostClassifier object at 0x7fb6a1af9d90> & SMOTE method & stepwise_feature_list,0.6793


In [ ]:
len(model_eval_test)

30

In [ ]:
model_eval_test

,Accuracy,Precision,F1,Recall
<catboost.core.CatBoostClassifier object at 0x7fb6a1af9250> & SMOTE method & rfe_features_list,0.9727,0.7099,0.7013,0.7099
LinearDiscriminantAnalysis(n_components=1) & SMOTE method & rfe_features_list,0.9443,0.8653,0.6790,0.8653
RandomForestClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.9766,0.7119,0.7213,0.7119
ExtraTreesClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.9756,0.6902,0.7031,0.6902
LGBMClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.9727,0.6249,0.6430,0.6249
GradientBoostingClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.9629,0.7261,0.6738,0.7261
<catboost.core.CatBoostClassifier object at 0x7fb6a1af9450> & SMOTE method & rfecv_features_list,0.9717,0.6669,0.6705,0.6669
LinearDiscriminantAnalysis(n_components=1) & SMOTE method & rfecv_features_list,0.9443,0.8653,0.6790,0.8653
RandomForestClassifier(random_state=1123) & SMOTE method & rfecv_features_list,0.9727,0.6674,0.6748,0.6674
ExtraTreesClassifier(random_state=1123) & SMOTE method & rfecv_features_list,0.9727,0.6674,0.6748,0.6674


# ADASYN

In [ ]:

# ADASYN 적용
def adasyn_eval(model, x_train, x_test, y_train, y_test, select_method):
  from imblearn.over_sampling import ADASYN
  adasyn = ADASYN(random_state=1123)


  X_train_ada, y_train_ada = adasyn.fit_resample(x_train, y_train)
  print('ADASYN 적용 전 학습용 피처/레이블 데이터 세트:', x_train.shape, y_train.shape)
  print('ADASYN 적용 후 학습용 피처/레이블 데이터 세트:', X_train_ada.shape, y_train_ada.shape)
  print('ADASYN 적용 후 레이블 값 분포: \n', pd.Series(y_train_ada).value_counts())
  
  clf = build_model(model, X_train_ada, y_train_ada) # 모델 학습


  # pred = model.predict(x_test)
  # pred_proba = model.predict_proba(x_test)
  # # cross_val_score(model, X_train_sm, y_train_sm, cv=skf, scoring='roc_auc').mean()


  # print('Confusion matrix')
  # print(confusion_matrix(y_test, pred))


  # print(f"Precision : {precision_score(y_test, pred, average='macro'):.4f}")
  # print(f"Recall    : {recall_score(y_test, pred, average='macro'):.4f}")
  # print(f"F1        : {f1_score(y_test, pred, average='macro'):.4f}")
  # print(f"Accuracy  : {accuracy_score(y_test, pred):.4f}")

  return model_eval_data(clf, X_train_ada, y_train_ada, x_test, y_test,  model_eval_train,
                                                                                     model_eval_test, Name = f'{model} & ADASYN method & {select_method}')

## ADASYN - 모든 모델 & feature selection 성능평가

In [ ]:
# feature selection 이후 파트 다시 정리하기

for select_method in feat_df.columns[1:]:
    
  cat_clf = CatBoostClassifier(random_state = 1123)
  lda_clf = LDA(n_components=1)
  RF_clf = RandomForestClassifier(random_state=1123)
  EX_clf = ExtraTreesClassifier(random_state=1123)
  LGB_clf = LGBMClassifier(random_state=1123)
  GB_clf = GradientBoostingClassifier(random_state=1123)

  models = [ ("CF", cat_clf),
            ("LDA", lda_clf),
            ("RF", RF_clf),
            ("EX", EX_clf),
            ("LGB", LGB_clf),
            ('GB', GB_clf)
          ]

  

  for model_name, model in models:

    # 데이터 초기화
    y_data = df['관리종목여부']
    x_data = df.drop(["관리종목여부"], axis=1)


    x_data_col = return_selected_data(feat_df, model_name, select_method) # 모델별 feature select 방식별 선정된 feature 반환
    # data augmentation 이전에 미리 원본 데이터로 테스트 데이터셋 구축(stratify 기능을 통해 균형 있게 테스트셋 구축)
    #' 당기순이익(손실)' 처리
    if ' 당기순이익(손실)' in x_data_col:
      idx3 = x_data_col.index(' 당기순이익(손실)')
    x_data_col[idx3] = '당기순이익'    

    if '유보액/납입자본비율' in x_data_col:
      idx4 = x_data_col.index('유보액/납입자본비율')
    x_data_col[idx4] = '유보액_납입자본비율'


    # feautre 선별
    new_x_data = x_data[x_data_col]
    new_y_data = y_data

    # 날짜 컬럼 빼주기
    if '날짜' in new_x_data.columns:
      new_x_data = new_x_data.drop('날짜', axis=1)   

    # stratify를 통해 test 검증셋과 증강할 데이터셋 구분
    x_train, x_test, y_train, y_test = train_test_split(new_x_data, new_y_data, test_size=0.2, random_state=1123, stratify=y_data) # 데이터를 균등하게 나누고 test 검증하기 위해
    
    model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test = adasyn_eval(model, x_train, x_test, y_train, y_test, select_method)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
178:	learn: 0.0490820	total: 1.08s	remaining: 4.96s
179:	learn: 0.0489679	total: 1.09s	remaining: 4.95s
180:	learn: 0.0487602	total: 1.09s	remaining: 4.95s
181:	learn: 0.0485620	total: 1.1s	remaining: 4.94s
182:	learn: 0.0483490	total: 1.1s	remaining: 4.93s
183:	learn: 0.0481047	total: 1.11s	remaining: 4.92s
184:	learn: 0.0478323	total: 1.11s	remaining: 4.91s
185:	learn: 0.0475460	total: 1.12s	remaining: 4.9s
186:	learn: 0.0473097	total: 1.13s	remaining: 4.9s
187:	learn: 0.0469636	total: 1.13s	remaining: 4.89s
188:	learn: 0.0466427	total: 1.14s	remaining: 4.88s
189:	learn: 0.0464035	total: 1.15s	remaining: 4.88s
190:	learn: 0.0462041	total: 1.15s	remaining: 4.88s
191:	learn: 0.0459765	total: 1.16s	remaining: 4.87s
192:	learn: 0.0455350	total: 1.16s	remaining: 4.86s
193:	learn: 0.0452608	total: 1.17s	remaining: 4.85s
194:	learn: 0.0448866	total: 1.17s	remaining: 4.84s
195:	learn: 0.0447034	total: 1.18s	remaining: 4.83s
196:	learn: 0.0445332	total: 1.1

In [ ]:
model_eval_test.sort_values('F1', ascending=False)

,Accuracy,Precision,F1,Recall
RandomForestClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.9766,0.7119,0.7213,0.7119
GradientBoostingClassifier(random_state=1123) & SMOTE method & select_k_feature_list,0.9766,0.7119,0.7213,0.7119
<catboost.core.CatBoostClassifier object at 0x7fb6cd87b610> & SMOTE method & perm_features_list,0.9775,0.6912,0.7138,0.6912
GradientBoostingClassifier(random_state=1123) & ADASYN method & select_k_feature_list,0.9688,0.7504,0.7063,0.7504
GradientBoostingClassifier(random_state=1123) & SMOTE method & perm_features_list,0.9707,0.7301,0.7040,0.7301
ExtraTreesClassifier(random_state=1123) & SMOTE method & rfe_features_list,0.9756,0.6902,0.7031,0.6902
<catboost.core.CatBoostClassifier object at 0x7fb6a1af9250> & SMOTE method & rfe_features_list,0.9727,0.7099,0.7013,0.7099
RandomForestClassifier(random_state=1123) & ADASYN method & rfe_features_list,0.9746,0.6897,0.6981,0.6897
ExtraTreesClassifier(random_state=1123) & ADASYN method & rfe_features_list,0.9736,0.6892,0.6933,0.6892
RandomForestClassifier(random_state=1123) & ADASYN method & rfecv_features_list,0.9736,0.6892,0.6933,0.6892


In [ ]:
len(model_eval_test)

60

# GAN

In [ ]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns  
import tensorflow as tf  
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Embedding  
from keras.layers import LeakyReLU
# from keras.layers.advanced_activations import LeakyReLU  
from keras.layers import ELU, PReLU, LeakyReLU
from tensorflow.keras.layers import concatenate 
from keras.models import Sequential, Model  
from keras.optimizers import Adam  
from keras.utils import to_categorical  
# from keras.layers.advanced_activations import LeakyReLU  
from keras.utils.vis_utils import plot_model  
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder  
import scipy.stats  
import datetime as dt  
import pydot  
import warnings  
warnings.filterwarnings("ignore")  
%matplotlib inline  

encoder = LabelEncoder()

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
# df= pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/real_final_data_for_modeling.csv')
# df.rename(columns={' 당기순이익(손실)':'당기순이익(손실)'}, inplace= True)
len(df)

5116

In [ ]:
df[df['관리종목여부']==1]

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,...,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률
191,226340,159.99,183.25,22.87,17.08,2.83,27.40,2.50,15.09,77.25,...,3,1,1,1,2018,83.330,1.04,24.0,0.000000,-0.775701
291,192410,-20.23,-15.04,-12.54,-8.49,-8.74,-46.47,-23.20,-22.56,66.27,...,3,1,0,0,2018,67.245,3.97,-85.0,0.000000,-1.243553
378,96690,-28.64,-11.85,-39.45,-4.98,-18.63,-56.72,-66.92,-17.89,98.12,...,3,1,0,0,2018,19.020,1.41,-232.0,0.000000,-11.545455
459,101000,-55.59,-51.75,-27.53,-15.89,-108.87,-658.43,-187.52,-27.37,36.19,...,3,1,0,0,2018,7.780,1.59,360.0,0.710000,0.747573
465,109740,-5.56,32.13,14.55,16.61,13.50,198.08,17.14,21.09,69.33,...,3,1,1,1,2018,24.470,2.39,423.0,0.480000,-0.711263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,14100,0.68,4.98,1.17,2.53,2.84,61.44,5.38,4.79,64.11,...,3,1,1,1,2021,1.270,0.61,3302.0,0.000000,11.007273
4956,222810,48.31,7.10,37.89,1.96,-17.76,-72.25,-89.73,9.91,61.45,...,3,1,1,0,2021,7.730,-329.00,0.0,-0.645474,-0.354859
4986,263920,-17.33,25.60,-1.61,-3.76,-12.62,-34.29,-43.12,-12.83,94.70,...,3,1,0,0,2021,93.860,0.69,-156.0,0.000000,-0.956558
5096,115530,1.88,6.70,58.91,-1.09,-1.00,-3.32,-0.84,-0.92,41.51,...,3,1,0,0,2021,3.780,-16.00,0.0,-0.829787,0.549708


In [ ]:
df.columns

Index(['Stock', '총자산증가율', '자기자본증가율', '매출액증가율', '총자본영업이익율', '총자본순이익율',
       '자본금순이익율', '매출액순이익율', '매출액영업이익율', '자기자본비율', '부채비율', '유보액_납입자본비율',
       '총자본회전율', '자기자본회전율', '당기순이익', '영업활동으로인한현금흐름', '성장성', '안정성', '활동성',
       '수익성', '현금흐름', 'Z_score', '부실기업판별_Z_score', '부도확률_O_score', 'F_score',
       'K_score', '부실기업판별_K_score', '자본잠식', '벌금', '불성실공시', '소송', '영업정지', '특허',
       '투자주의환기종목', '업종', '관리종목여부', '관리종목요건_감사의견', '관리종목요건_매출액', '관리종목요건_영업손실',
       '관리종목요건_법인세비용차감전계속사업손실', '날짜', 'PER', 'PBR', 'EPS', 'DIV', 'EPS성장률'],
      dtype='object')

In [ ]:
# df= pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/real_final_data_for_modeling.csv')
# df.rename(columns={' 당기순이익(손실)':'당기순이익'}, inplace= True)
# df.rename(columns={'유보액/납입자본비율':'유보액_납입자본비율'}, inplace =True)
# # df.rename(columns={' 수익성':'수익성'}, inplace=True)



# # 카테고리컬인 애들은 다 astype으로 미리 바꿔주자 >> 다 int64임 
# df[['성장성', '안정성', '활동성','수익성', '현금흐름','부실기업판별_Z_score','F_score','부실기업판별_K_score', '자본잠식', '벌금', '불성실공시', '소송', '영업정지', '특허',
#        '투자주의환기종목', '업종','관리종목요건_감사의견', '관리종목요건_매출액', '관리종목요건_영업손실', '관리종목요건_법인세비용차감전계속사업손실']] = df[['성장성', '안정성', '활동성','수익성', '현금흐름','부실기업판별_Z_score','F_score','부실기업판별_K_score', '자본잠식', '벌금', '불성실공시', '소송', '영업정지', '특허',
#        '투자주의환기종목', '업종','관리종목요건_감사의견', '관리종목요건_매출액', '관리종목요건_영업손실', '관리종목요건_법인세비용차감전계속사업손실']].astype('int64')
# df['관리종목여부'] = df['관리종목여부'].astype('int64') 


In [ ]:
# selectioned_flist = ['총자산증가율', '자기자본증가율', '매출액증가율', '총자본영업이익율', '자본금순이익율', '매출액영업이익율', '부채비율', '총자본회전율', '당기순이익', '영업활동으로인한현금흐름', 'Z_score', '부도확률_O_score', 'K_score', '자본잠식', '투자주의환기종목', '날짜', 'PER', 'PBR', 'EPS', 'DIV', 'EPS성장률']

In [ ]:
def gan_before(selectioned_flist):
  # 데이터 준비
  df_minority_data = df.loc[df['관리종목여부']==1] # 타겟==1인 애들만
  df_minority_data_withouttv = df_minority_data.loc[:, df_minority_data.columns != '관리종목여부'] # 타겟값을 제외한 전체
  
  # numberical_df, categorical_df 만들어주기
  numerical_list = []
  categorical_list = []
  globals()['numcat_list'] = [] # 나중에 쓰일 곳이 있다. -> 나중에 통합변수로 쓰나보다.

  for i in selectioned_flist:
    if df_minority_data_withouttv[i].dtypes =='float64': # numerical
      numerical_list.append(i)
      globals()['numcat_list'].append(i) # 이 부분 살짝 수정
      globals()['numerical_df'] = df_minority_data_withouttv[numerical_list] # 수치형만 여기에다

    else:
      categorical_list.append(i) 
      globals()['numcat_list'].append(i)
      globals()['categorical_df'] = df_minority_data_withouttv[categorical_list]




  # 수치형 만들어주기
  globals()['generator_input_list'] = []
  for i in globals()['numerical_df'].columns:
    globals()[f'{i}_numerical'] = pd.DataFrame(numerical_df[i]) # 이 부분도 살짝 수정
    globals()['generator_input_list'].append(eval(f'{i}_numerical.shape[1]')) # generator_input_list에서 1을 반환

  # 범주형 만들어주기
  for i in globals()['categorical_df'].columns:
    globals()[f'{i}_categorical'] = pd.DataFrame(categorical_df[i]) # shape를 찍기 위해??
    generator_input_list.append(eval(f'{i}_categorical.shape[1]'))








  ## GAN TRAIN INPUT LIST 만들기

  # gantrain_input_list 수치형 만들어주기
  globals()['gantrain_input_list'] = []
  for i in globals()['numerical_df'].columns: # 수치형 df 컬럼 돌면서
    gantrain_input_list.append(eval(f'{i}_numerical.values'))

  # gantrain_input_list 범주형 만들어주기
  for i in globals()['categorical_df'].columns:
    gantrain_input_list.append(eval(f'{i}_categorical.values'))

  

  # 이건 뭘까
  globals()['numerical_catsh_list'] = []
  categorical_catsh_list = [] # 확인용
  globals()['all_catsh_list'] = []
  for i in range(1, len(globals()['numerical_df'].columns)+1): ################????????????? 이거 왜 1부터 돌아??????????????????????? numerical에서 n-1개 넣는다.
    globals()['numerical_catsh_list'].append(f'catsh{i}')  # catsh는 비어있는 그냥 변수
    all_catsh_list.append(f'catsh{i}')

  # categorical
  for i in range(len(globals()[f'numerical_catsh_list'])+1, len(globals()['categorical_df'].columns)+len(globals()[f'numerical_catsh_list'])+1):   # len(numerical_catsh_list)+1 
    categorical_catsh_list.append(f'catsh{i}')
    all_catsh_list.append(f'catsh{i}')

  return generator_input_list
  


  

In [ ]:
# # gan 데이터 사전작업 준비
# gan_before(selectioned_flist)


# print(f'selected_feature 개수 : {len(selectioned_flist)}')
# print(f'all_catsh_list 개수: {len(all_catsh_list)}')
# print(f'all_catsh_list : {all_catsh_list}')
# print()

# print(f'generator_input_list : {gan_before(selectioned_flist)}')
# print(len(gan_before(selectioned_flist)))

selected_feature 개수 : 21
all_catsh_list 개수: 21
all_catsh_list : ['catsh1', 'catsh2', 'catsh3', 'catsh4', 'catsh5', 'catsh6', 'catsh7', 'catsh8', 'catsh9', 'catsh10', 'catsh11', 'catsh12', 'catsh13', 'catsh14', 'catsh15', 'catsh16', 'catsh17', 'catsh18', 'catsh19', 'catsh20', 'catsh21']

generator_input_list : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
21


In [ ]:
def gan_generator(): 
    # numerical hidden 
    for i in range( 1, len(globals()['numerical_df'].columns)+1):  # numerical 안막을거면 +2 해줘야 함! 
      if i == 1:
        noise = Input(shape = (3902 ,))     # 관리종목여부 ==0인 기업이 5000개이므로 4884개의 가상 데이터 생성 ㄱㄱ (1:1 비율로!) / try2: 3902
        globals()[f'hidden_{i}'] = Dense(8, kernel_initializer = "he_uniform")(noise)    
        globals()[f'hidden_{i}'] = LeakyReLU(0.2)(globals()[f'hidden_{i}'])    
        globals()[f'hidden_{i}'] = BatchNormalization(momentum = 0.8)(globals()[f'hidden_{i}'])
      else: 
        globals()[f'hidden_{i}'] = Dense(8, kernel_initializer = "he_uniform")(globals()[f'hidden_{i-1}'])    
        globals()[f'hidden_{i}'] = LeakyReLU(0.2)(globals()[f'hidden_{i}'])    
        globals()[f'hidden_{i}'] = BatchNormalization(momentum = 0.8)(globals()[f'hidden_{i}'])

        
    # categorical hidden 
    for i in range(len(globals()['numerical_df'].columns)+1, len(globals()['categorical_df'].columns)+1+len(globals()['numerical_df'].columns)):  
      
      globals()[f'hidden_{i}'] = Dense(8, kernel_initializer = "he_uniform")(globals()[f'hidden_{i-1}'])    
      globals()[f'hidden_{i}'] = LeakyReLU(0.2)(globals()[f'hidden_{i}'])    
      globals()[f'hidden_{i}'] = BatchNormalization(momentum = 0.8)(globals()[f'hidden_{i}'])
    
    
    
    
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    # numerical branch
    for i in range(1, len(globals()['numerical_df'].columns)+1):

      globals()[f'branch_{i}'] = Dense(32, kernel_initializer = "he_uniform")(globals()[f'hidden_{i}'])    
      globals()[f'branch_{i}'] = LeakyReLU(0.2)(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = BatchNormalization(momentum = 0.8)(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = Dense(64, kernel_initializer = "he_uniform")(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = LeakyReLU(0.2)(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = BatchNormalization(momentum=0.8)(globals()[f'branch_{i}']) 
  
    # categorical branch
    for i in range(len(globals()['numerical_df'].columns)+1, len(globals()['categorical_df'].columns)+1+len(globals()['numerical_df'].columns)):   

      globals()[f'branch_{i}'] = Dense(32, kernel_initializer = "he_uniform")(globals()[f'hidden_{i}'])    
      globals()[f'branch_{i}'] = LeakyReLU(0.2)(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = BatchNormalization(momentum = 0.8)(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = Dense(64, kernel_initializer = "he_uniform")(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = LeakyReLU(0.2)(globals()[f'branch_{i}'])    
      globals()[f'branch_{i}'] = BatchNormalization(momentum=0.8)(globals()[f'branch_{i}'])
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    # numerical
    # for i in range(1,len(globals()['numerical_df'].columns)+1): # 손보영 이부분 틀림
    #   for a in globals()['numerical_df'].columns:
    #     globals()[f'catsh{i}'] = globals()[f'{a}_numerical'].shape[1]
    # # categorical
    # for i in range(len(globals()['numerical_df'].columns)+1,len(globals()['categorical_df'].columns)+1+len(globals()['numerical_df'].columns)): 
    #   for a in globals()['categorical_df'].columns:
    #     globals()[f'catsh{i}'] = globals()[f'{a}_categorical'].shape[1]
    
    # numerical
    cnt = 1
    for a in globals()['numerical_df'].columns:
      globals()[f'catsh{cnt}'] = globals()[f'{a}_numerical'].shape[1]
      cnt+=1
    
    # categorical
    for a in globals()['categorical_df'].columns:
      globals()[f'catsh{cnt}'] = globals()[f'{a}_categorical'].shape[1]
      cnt+=1

    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    # numerical
    all_branchoutput_list = []
    numerical_branchoutput_list = [] # 확인용 
    for i in range(1, len(globals()['numerical_df'].columns)+1):
      globals()[f'branch_{i}_output'] = Dense(globals()[f'catsh{i}'], activation = "swish")(globals()[f'branch_{i}'])   # globals()[f'catsh{i}']  / globals()[f'branch_{i}'] # softmax
      numerical_branchoutput_list.append(f'branch_{i}_output')
      all_branchoutput_list.append(f'branch_{i}_output')
    # categorical
    categorical_branchoutput_list = [] # 확인용  
    for i in range(len(globals()['numerical_df'].columns)+1, len(globals()['categorical_df'].columns)+1+len(globals()['numerical_df'].columns)):
      globals()[f'branch_{i}_output'] = Dense(globals()[f'catsh{i}'],activation = "hard_sigmoid" )(globals()[f'branch_{i}'])   # globals()[f'catsh{i}']  / globals()[f'branch_{i}'] # softmax // 
      categorical_branchoutput_list.append(f'branch_{i}_output')
      all_branchoutput_list.append(f'branch_{i}_output')
    # return numerical_branchoutput_list , categorical_branchoutput_list , all_branchoutput_list 
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    # combined_output = concatenate([branch_1_output, branch_2_output, branch_3_output,branch_4_output,branch_5_output,branch_6_output,branch_7_output,branch_8_output,branch_9_output,branch_10_output,branch_11_output,branch_12_output,branch_13_output])
    combined_output = branch_1_output
    for i in range(2,len(all_branchoutput_list)+1):
      combined_output = concatenate([combined_output,globals()[f'branch_{i}_output']])  # 
    return  Model(inputs = noise, outputs = combined_output) 

In [ ]:
# generator = gan_generator() # 손보영 또 틀림
# generator.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 3902)]       0           []                               
                                                                                                  
 dense_90 (Dense)               (None, 8)            31224       ['input_3[0][0]']                
                                                                                                  
 leaky_re_lu_68 (LeakyReLU)     (None, 8)            0           ['dense_90[0][0]']               
                                                                                                  
 batch_normalization_63 (BatchN  (None, 8)           32          ['leaky_re_lu_68[0][0]']         
 ormalization)                                                                              

In [ ]:
def gan_discriminator(inputs_n):
  # Input from generator
  d_input = Input(shape = (inputs_n,))
  d = Dense(128, kernel_initializer='he_uniform')(d_input)
  d = LeakyReLU(0.2)(d)
  d = Dense(64, kernel_initializer = 'he_uniform')(d)
  d = LeakyReLU(0.2)(d)
  d = Dense(32, kernel_initializer='he_uniform')(d)
  d = LeakyReLU(0.2)(d)
  d = Dense(16, kernel_initializer='he_uniform')(d)
  d = LeakyReLU(0.2)(d)
  d = Dense(8, kernel_initializer='he_uniform')(d)
  d = LeakyReLU(0.2)(d)

  # Output Layer
  d_output = Dense(1, activation = 'sigmoid')(d) 

  # compile and return model
  model = Model(inputs = d_input, outputs = d_output)
  model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=0.0002, beta_1 = 0.5)) 
  return model



In [ ]:
# inputs_n = sum(globals()['generator_input_list'])

# discriminator = gan_discriminator(inputs_n)
# discriminator.summary()

In [ ]:
def define_complete_gan(generator, discriminator):

  gan_output = discriminator(generator.output)
  # Initialize gan
  model = Model(inputs = generator.input, outputs = gan_output)
  # Model Compilation
  model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=0.0002, beta_1=0.5))
  discriminator.trainable = True # 학습 막고 싶으면 false
  return model



In [ ]:
# completegan = define_complete_gan(generator, discriminator)
# completegan.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 3902)]       0           []                               
                                                                                                  
 dense_90 (Dense)               (None, 8)            31224       ['input_3[0][0]']                
                                                                                                  
 leaky_re_lu_68 (LeakyReLU)     (None, 8)            0           ['dense_90[0][0]']               
                                                                                                  
 batch_normalization_63 (BatchN  (None, 8)           32          ['leaky_re_lu_68[0][0]']         
 ormalization)                                                                              

In [ ]:
def gan_train(gan, generator, discriminator, latent_dim, n_epochs, n_batch, n_eval): 
  half_batch = int(n_batch/2) # half_batch는 왜하는 걸까
  discriminator_loss = []
  generator_loss = []
  valid = np.ones((half_batch, 1))
  y_gan = np.ones((n_batch, 1))
  fake = np.zeros((half_batch, 1))
  # n_epochs = 10000 # 나중에 여러 에포크로 시도해보기
  for i in range(n_epochs): # ----------------> 이부분 다시 봐보자
    # select random batch from real categorical and numerical data
    idx = np.random.randint(0, 116, half_batch) ###################################################################### 이부분 0 ~ 116까지 수 범위로 n_epochs//2 만큼 배열 채우기
    
    
    concatenate_list = []
    # categorical222_list = []
    # for num, j in enumerate(globals()['numcat_list']): # 모든 컬럼 이름을 모아놓은 것.
    #   if j in list(globals()['numerical_df'].columns): #numerical 변수면
    #     globals()[f'{j}_real'] = globals()[f'{j}_numerical'].values[idx]
    #     concatenate_list.append( globals()[f'{j}_real'])
    #   else: # 카테고리컬 변수면
    #     globals()[f'{j}_real'] = globals()[f'{j}_categorical'].values[idx]
    #     categorical222_list.append( globals()[f'{j}_real'])
    
    for j in globals()['numerical_df'].columns:
      globals()[f'{j}_real'] = globals()[f'{j}_numerical'].values[idx]
      concatenate_list.append(globals()[f'{j}_real'])

    for j in globals()['categorical_df'].columns:
      globals()[f'{j}_real'] = globals()[f'{j}_categorical'].values[idx]
      concatenate_list.append(globals()[f'{j}_real'])   


    
    # concatenate_list.extend(categorical222_list)

    real_data = np.concatenate(concatenate_list, axis=1)

    #generate fake samples from the noise
    noise = np.random.normal(0,1, (half_batch, latent_dim)) ############################ latent_dim은 어따가 쓰는겨??
    fake_data = generator.predict(noise)

    #train the discriminator and return losses and acc
    d_loss_real = da_real = discriminator.train_on_batch(real_data, valid)
    d_loss_fake = da_fake = discriminator.train_on_batch(fake_data, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator_loss.append(d_loss)

    #generate noise for generator input and train the generator (to have the discriminator label samples as valid)
    noise = np.random.normal(0, 1, (n_batch, latent_dim))
    g_loss = gan.train_on_batch(noise, y_gan)
    generator_loss.append(g_loss)


    #evaluate progress
    if (i+1) % n_eval == 0:  
        print ("Epoch: %d [Discriminator loss: %f] [Generator loss: %f]" % (i + 1, d_loss, g_loss ))  
          

  # 시각화 하고싶으면 열어~~~~        
  # plt.figure(figsize = (20, 10))  
  # plt.plot(generator_loss, label = "Generator loss")  
  # plt.plot(discriminator_loss, label = "Discriminator loss")     
  # plt.title("Stats from training GAN")  
  # plt.grid()  
  # plt.legend() 
    

In [ ]:
# # 자동화로 갠 트레인 가보자잇~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# latent_dim = 3902
# gan_train(completegan, generator, discriminator,latent_dim,n_epochs = 7000,n_batch = 63 , n_eval = 200)

In [ ]:
# 아래 코드 자동화
def gan_afteone():
  noise = np.random.normal(0, 1, (3902, 3902))
  generated_mixed_data = generator.predict(noise)

  col_name = list(numerical_df.columns) + list(categorical_df.columns)

  globals()['mixed_gen_df'] = pd.DataFrame(data = generated_mixed_data, columns = col_name)


  ######################################## 시각화할거면 넣어~~ ####################################3
  # original_df = pd.DataFrame()
  # for i in globals()['numerical_df'].columns:
  #   original_df = pd.concat([original_df, globals()[f'{i}_numerical']], axis=1)
  # for i in globals()['categorical_df'].columns:
  #   original_df = pd.concat([original_df, globals()[f'{i}_categorical']], axis=1)


  # def normal_distribution(org, noise):  
  #     org_x = np.linspace(org.min(), org.max(), len(org))  
  #     noise_x = np.linspace(noise.min(), noise.max(), len(noise))  
  #     org_y = scipy.stats.norm.pdf(org_x, org.mean(), org.std())  
  #     noise_y = scipy.stats.norm.pdf(noise_x, noise.mean(), noise.std())  
  #     n, bins, patches = plt.hist([org, noise], density = True, alpha = 0.5, color = ["green", "red"])  
  #     xmin, xmax = plt.xlim()  
  #     plt.plot(org_x, org_y, color = "green", label = "Original data", alpha = 0.5)  
  #     plt.plot(noise_x, noise_y, color = "red", label = "Generated data", alpha = 0.5)  
  #     title = f"Original data mean {np.round(org.mean(), 4)}, Original data std {np.round(org.std(), 4)}, Original data var {np.round(org.var(), 4)}\nGenerated data mean {np.round(noise.mean(), 4)}, Generated data {np.round(noise.std(), 4)}, Generated data var {np.round(noise.var(), 2)}"
  #     plt.title(title)  
  #     plt.legend()  
  #     plt.grid()  
  #     plt.show()  
  #     plt.close()

  # Numeric_columns=numerical_df.columns  

  # for column in numerical_df.columns:  
  #     print(column, "Comparison between Original Data and Generated Data")  
  #     normal_distribution(original_df[column], mixed_gen_df[column])

  



In [ ]:

def return_gan_data(): # gan_data 뱉어내기(학습용 데이터)
  def binary_step(data):
    # for i in range(len(data)): 
      if data<0.5:
        return 0
      else:
        return 1

  for i in globals()['categorical_df'].columns:
    globals()['mixed_gen_df'][f'{i}'] = globals()['mixed_gen_df'][f'{i}'].apply(binary_step) # 0,1로 다 바꿔주기 
    globals()['mixed_gen_df'][f'{i}'] = globals()['mixed_gen_df'][f'{i}'].astype('int64') # type변환 

  # df_generated_data 생성
  ## numerical_data >> generated_data에 넣어주기 
  df_generated_data = pd.DataFrame()   
  for i in globals()['numerical_df'].columns:
    df_generated_data[f'{i}'] = globals()['mixed_gen_df'][f'{i}']

  ## categorical_data >> generated_data에 넣어주기 
  for i in globals()['categorical_df'].columns:
    df_generated_data[f'{i}'] = globals()['mixed_gen_df'][f'{i}']  
  ## 타겟값 붙여주기 
  df_generated_data['관리종목여부'] = 1
  # 이 다음이 원본데이터에서 모델학습에 쓸 피쳐들 따로 떼서 final_df 만들어주는거 >> 이건 자동화 밖에서 이미 설정해줄거라 포함안시켜도 될 듯 
  
  # 생성된 데이터 X1, y1으로 나뉘어주고 
  y1 = df_generated_data['관리종목여부']
  X1 = df_generated_data.drop(["관리종목여부"], axis=1)


  

  return X1,  y1 # gan을 이용한 증강 데이터 반환(학습용 데이터로 쓸거임.)

In [ ]:
globals()['mixed_gen_df']

,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,자본금순이익율,매출액영업이익율,부채비율,총자본회전율,당기순이익,영업활동으로인한현금흐름,Z_score,부도확률_O_score,K_score,PER,PBR,EPS,DIV,EPS성장률,자본잠식,투자주의환기종목
0,-0.154903,-0.001979,0.045348,-0.273571,-3.050110e-02,-0.258725,-0.000006,1.813171,-0.000477,1.909786e+01,1.935377,-0.070183,0.951260,-0.203488,1.191203,-0.053856,-0.000174,-0.094217,1,0
1,4.878868,17.356112,0.673246,11.755629,7.632760e+00,26.214933,18.481697,-0.004940,-0.000067,-8.648976e-04,-0.056914,0.503370,-0.253246,-0.102925,4.881054,8.686899,34.653313,11.177094,1,1
2,3.625801,-0.234281,-0.220515,5.165982,-1.891163e-03,11.207108,-0.006430,1.135094,3.104252,1.677025e+01,2.973492,-0.162731,-0.205016,-0.278081,-0.095033,-0.188588,-0.142886,-0.001278,0,0
3,5.542056,12.073199,3.765460,8.750520,5.900155e+00,30.247942,8.566843,-0.004464,-0.002335,-1.587062e-08,-0.043107,7.063031,-0.234432,-0.154227,6.065943,4.995232,24.798471,14.116750,1,1
4,11.000465,0.872773,0.426237,-0.023072,3.309167e+01,-0.000035,-0.040269,0.104360,6.566389,2.047413e+01,-0.261035,-0.069988,-0.257135,-0.278463,11.820292,-0.128151,5.715765,13.039199,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897,5.516299,-0.230230,0.378008,-0.070830,4.556764e+00,0.007470,-0.000104,-0.202862,6.289618,2.061005e+01,2.383807,-0.271889,-0.244557,-0.277028,-0.085468,0.468865,-0.101359,-0.001354,1,1
3898,3.692073,-0.000008,-0.259939,5.274873,-8.606061e-06,18.268843,0.477337,0.698971,13.684101,-2.604991e-01,1.537495,2.424362,-0.274813,-0.270393,-0.053666,1.823946,3.484244,-0.278386,1,1
3899,8.211337,7.162372,0.865584,5.129210,-8.770260e-09,-0.261282,21.932917,-0.260154,-0.000008,2.399985e+01,-0.164198,-0.157556,-0.050654,2.594426,3.767407,0.430121,6.899855,6.913964,1,1
3900,-0.126412,2.198990,-0.161123,8.355503,-2.894790e-03,20.102497,1.429287,0.532216,12.819090,-5.400331e-02,1.026370,2.280413,-0.274660,-0.273694,-0.058379,1.826844,3.102669,-0.275477,1,1


In [ ]:

# y = df['관리종목여부']
# X = df[selectioned_flist] # 이거 함수 자동화
# gan_afteone()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1123, stratify=y)
# gan_X, gan_y = return_gan_data() # 타겟값 1인 데이터 3902개씩 만들기

# new_train_X = pd.concat([X_train, gan_X], axis=0)
# new_train_X = new_train_X.reset_index(drop=True) # 인덱스 재배열

# new_train_y = np.concatenate((y_train, gan_y))



# model = CatBoostClassifier(random_state = 1123)
# clf = build_model(model, new_train_X, new_train_y)
# pred = clf.predict(X_test)


# print(f"Precision : {precision_score(y_test, pred, average='macro'):.4f}")
# print(f"Recall    : {recall_score(y_test, pred, average='macro'):.4f}")
# print(f"F1        : {f1_score(y_test, pred, average='macro'):.4f}")
# print(f"Accuracy  : {accuracy_score(y_test, pred):.4f}")


Learning rate set to 0.025027
0:	learn: 0.6279683	total: 8.79ms	remaining: 8.78s
1:	learn: 0.5634014	total: 15.8ms	remaining: 7.9s
2:	learn: 0.5090673	total: 20.8ms	remaining: 6.92s
3:	learn: 0.4580647	total: 27.7ms	remaining: 6.9s
4:	learn: 0.4165380	total: 33.3ms	remaining: 6.62s
5:	learn: 0.3820050	total: 38.8ms	remaining: 6.42s
6:	learn: 0.3437832	total: 44.8ms	remaining: 6.36s
7:	learn: 0.3145023	total: 51.4ms	remaining: 6.37s
8:	learn: 0.2897614	total: 57.6ms	remaining: 6.34s
9:	learn: 0.2660854	total: 63.4ms	remaining: 6.28s
10:	learn: 0.2449011	total: 68.6ms	remaining: 6.16s
11:	learn: 0.2237598	total: 74ms	remaining: 6.09s
12:	learn: 0.2031553	total: 80.9ms	remaining: 6.14s
13:	learn: 0.1841646	total: 86.5ms	remaining: 6.09s
14:	learn: 0.1704924	total: 90.8ms	remaining: 5.96s
15:	learn: 0.1595739	total: 97ms	remaining: 5.96s
16:	learn: 0.1480070	total: 102ms	remaining: 5.91s
17:	learn: 0.1355363	total: 106ms	remaining: 5.78s
18:	learn: 0.1268518	total: 112ms	remaining: 5.76s
1

In [ ]:
sum(globals()['generator_input_list'])

21

## GAN - 모든 모델 & feature selection 성능평가

In [ ]:
# def gan_eval(model, X_train, X_test, y_train, y_test, select_method):
#     # gan 관련 함수 실행
#     gan_before(list(X_train.columns))
#     generator = gan_generator()
#     inputs_n = sum(globals()['generator_input_list']) 
#     discriminator = gan_discriminator(inputs_n)
#     completegan = define_complete_gan(generator, discriminator)
#     latent_dim = 3902
#     gan_train(completegan, generator, discriminator,latent_dim,n_epochs = 2000,n_batch = 63 , n_eval = 200) ####################### 이거 나중에 바꿔줘
#     gan_afteone() # mixed 데이터 만들기

#     # gan으로 학습데이터 만들기
#     gan_X, gan_y = return_gan_data() # 타겟값 1인 데이터 3902개씩 만들기

#     ############################################################# concat 이전에 컬럼 형태 맞춰줘야한다!!!!!!!!!!!!!!!!!!!!! 
#     gan_X = gan_X[X_train.columns]

#     # 기존 train 데이터에 합치기
#     new_train_X = pd.concat([X_train, gan_X], axis=0)
#     new_train_X = new_train_X.reset_index(drop=True) # 인덱스 재배열


#     new_train_y = np.concatenate((y_train, gan_y))

#     # 학습
#     clf = build_model(model, new_train_X, new_train_y) # 모델 학습


#     # 여기서 X_test 컬럼 순서를 new_train_X 컬럼으로 맞춰줘야한다.!!! 왜냐면 gan 돌릴때 순서가 numerical 다음 categorical 변수가 나오므로
#     X_test = X_test[new_train_X.columns]

#     return  model_eval_data(clf, new_train_X, new_train_y, X_test, y_test, model_eval_train, model_eval_test, Name = f'{model} & GAN method & {select_method}')

In [ ]:



# 데이터 초기화
df2= pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/real_final_data_for_modeling.csv')
df2.rename(columns={' 당기순이익(손실)':'당기순이익'}, inplace= True)
df2.rename(columns={'유보액/납입자본비율':'유보액_납입자본비율'}, inplace =True)
# df.rename(columns={' 수익성':'수익성'}, inplace=True)


# 카테고리컬인 애들은 다 astype으로 미리 바꿔주자 >> 다 int64임 
df2[['성장성', '안정성', '활동성','수익성', '현금흐름','부실기업판별_Z_score','F_score','부실기업판별_K_score', '자본잠식', '벌금', '불성실공시', '소송', '영업정지', '특허',
      '투자주의환기종목', '업종','관리종목요건_감사의견', '관리종목요건_매출액', '관리종목요건_영업손실', '관리종목요건_법인세비용차감전계속사업손실']] = df2[['성장성', '안정성', '활동성','수익성', '현금흐름','부실기업판별_Z_score','F_score','부실기업판별_K_score', '자본잠식', '벌금', '불성실공시', '소송', '영업정지', '특허',
      '투자주의환기종목', '업종','관리종목요건_감사의견', '관리종목요건_매출액', '관리종목요건_영업손실', '관리종목요건_법인세비용차감전계속사업손실']].astype('int64')
df2['관리종목여부'] = df2['관리종목여부'].astype('int64') 



for select_method in feat_df.columns[1:]:

  cat_clf = CatBoostClassifier(random_state = 1123)
  lda_clf = LDA(n_components=1)
  RF_clf = RandomForestClassifier(random_state=1123)
  EX_clf = ExtraTreesClassifier(random_state=1123)
  LGB_clf = LGBMClassifier(random_state=1123)
  GB_clf = GradientBoostingClassifier(random_state=1123)

  models = [ ("CF", cat_clf),
            ("LDA", lda_clf),
            ("RF", RF_clf),
            ("EX", EX_clf),
            ("LGB", LGB_clf),
            ('GB', GB_clf)
          ]


  for model_name, model in models:
    
    df=df2.copy()


    x_data_col = return_selected_data(feat_df, model_name, select_method) # 모델별 feature select 방식별 선정된 feature 반환
    # data augmentation 이전에 미리 원본 데이터로 테스트 데이터셋 구축(stratify 기능을 통해 균형 있게 테스트셋 구축)
    #' 당기순이익(손실)' 처리
    if ' 당기순이익(손실)' in x_data_col:
      idx3 = x_data_col.index(' 당기순이익(손실)')
      x_data_col[idx3] = '당기순이익'

    if '유보액/납입자본비율' in x_data_col:
      idx4 = x_data_col.index('유보액/납입자본비율')
      x_data_col[idx4] = '유보액_납입자본비율'

    # feautre 선별
    X = df[x_data_col]
    y = df['관리종목여부']

    # 날짜 컬럼 빼주기
    if '날짜' in list(X.columns):
      X = X.drop('날짜', axis=1)

    
    # 미리 테스트셋 만들기
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1123, stratify=y)


    # gan 관련 함수 실행
    gan_before(list(X_train.columns))
    generator = gan_generator()
    inputs_n = sum(globals()['generator_input_list']) 
    discriminator = gan_discriminator(inputs_n)
    completegan = define_complete_gan(generator, discriminator)
    latent_dim = 3902
    gan_train(completegan, generator, discriminator,latent_dim,n_epochs = 2000,n_batch = 63 , n_eval = 200) ####################### 이거 나중에 바꿔줘
    gan_afteone() # mixed 데이터 만들기

    # gan으로 학습데이터 만들기
    gan_X, gan_y = return_gan_data() # 타겟값 1인 데이터 3902개씩 만들기

    ############################################################# concat 이전에 컬럼 형태 맞춰줘야한다!!!!!!!!!!!!!!!!!!!!! 
    gan_X = gan_X[X_train.columns]

    # 기존 train 데이터에 합치기
    new_train_X = pd.concat([X_train, gan_X], axis=0)
    new_train_X = new_train_X.reset_index(drop=True) # 인덱스 재배열


    new_train_y = np.concatenate((y_train, gan_y))

    # 학습
    clf = build_model(model, new_train_X, new_train_y) # 모델 학습


    # 여기서 X_test 컬럼 순서를 new_train_X 컬럼으로 맞춰줘야한다.!!! 왜냐면 gan 돌릴때 순서가 numerical 다음 categorical 변수가 나오므로
    X_test = X_test[new_train_X.columns]

    model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test = model_eval_data(clf, new_train_X, new_train_y, X_test, y_test, model_eval_train, model_eval_test, Name = f'{model} & GAN method & {select_method}')
    
    


Epoch: 200 [Discriminator loss: 4.108914] [Generator loss: 0.006896]
Epoch: 400 [Discriminator loss: 3.862556] [Generator loss: 0.001253]
Epoch: 600 [Discriminator loss: 4.038361] [Generator loss: 0.003999]
Epoch: 800 [Discriminator loss: 4.619692] [Generator loss: 0.003094]
Epoch: 1000 [Discriminator loss: 4.304672] [Generator loss: 0.002577]
Epoch: 1200 [Discriminator loss: 4.730796] [Generator loss: 0.001184]
Epoch: 1400 [Discriminator loss: 5.543006] [Generator loss: 0.000723]
Epoch: 1600 [Discriminator loss: 6.187572] [Generator loss: 0.001613]
Epoch: 1800 [Discriminator loss: 6.877066] [Generator loss: 0.000274]
Epoch: 2000 [Discriminator loss: 7.590446] [Generator loss: 0.000272]
Learning rate set to 0.025027
0:	learn: 0.6277734	total: 6.33ms	remaining: 6.32s
1:	learn: 0.5711394	total: 11.4ms	remaining: 5.67s
2:	learn: 0.5173836	total: 16.1ms	remaining: 5.34s
3:	learn: 0.4692577	total: 21ms	remaining: 5.22s
4:	learn: 0.4319359	total: 26.6ms	remaining: 5.29s
5:	learn: 0.3946532	t

False

In [ ]:
# 맨처음 컬럼개수 20개
# 그다음 컬럼개수 21개

In [ ]:
len(X_train.columns)

20

In [ ]:
X.shape

(5116, 20)

In [ ]:
X_train.shape

(4092, 20)

In [ ]:
inputs_n

21

In [ ]:
gan_X.columns # 왜 생성이 안됐을까

Index(['총자산증가율', '자기자본증가율', '매출액증가율', '총자본영업이익율', '자본금순이익율', '매출액영업이익율',
       '부채비율', '총자본회전율', '당기순이익', '영업활동으로인한현금흐름', 'Z_score', '부도확률_O_score',
       'K_score', '자본잠식', '투자주의환기종목', 'PER', 'PBR', 'EPS', 'DIV', 'EPS성장률'],
      dtype='object')

In [ ]:
model_eval_test.to_csv('/content/drive/MyDrive/핀테크 파이널/eval_test.csv', index=False)

In [ ]:
model_eval_test.tail(5)

,Accuracy,Precision,F1,Recall
RandomForestClassifier(random_state=1123) & ADASYN method & select_k_feature_list,0.9775,0.6062,0.6458,0.6062
ExtraTreesClassifier(random_state=1123) & ADASYN method & select_k_feature_list,0.9756,0.6264,0.6559,0.6264
LGBMClassifier(random_state=1123) & ADASYN method & select_k_feature_list,0.9736,0.5830,0.6076,0.5830
GradientBoostingClassifier(random_state=1123) & ADASYN method & select_k_feature_list,0.9688,0.7504,0.7063,0.7504
<catboost.core.CatBoostClassifier object at 0x7fb63305ee50> & GAN method & rfe_features_list,0.9795,0.5860,0.6327,0.5860


In [ ]:
X_train

,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,자본금순이익율,매출액영업이익율,부채비율,총자본회전율,당기순이익,영업활동으로인한현금흐름,Z_score,부도확률_O_score,K_score,자본잠식,투자주의환기종목,PER,PBR,EPS,DIV,EPS성장률
3946,51.08,-33.46,-48.06,-15.80,-220.25,-2804.73,144.75,0.01,-1.479115e+10,-6.040461e+09,2.423710,0.004008,-6.172493,0,0,33.855,7.06,-1099.0,0.000000,-3.303983
3704,19.68,5.23,6.36,5.30,29.90,3.40,55.56,1.56,2.129262e+09,6.450040e+08,4.158521,0.000314,27.000318,0,0,25.600,1.60,142.0,2.340000,-0.759729
1542,69.71,129.13,1.76,3.56,-292.00,3.73,23.56,0.96,-1.180134e+10,2.859824e+09,10.550479,0.010310,11.189354,0,0,173.750,1.07,12.0,0.000000,11.000000
457,-4.66,-5.13,-11.62,-8.25,-34.91,-16.57,15.73,0.50,-2.363712e+09,1.197800e+07,4.384619,0.000395,17.898521,1,0,41.360,0.69,-175.0,0.000000,-0.356618
436,11.58,12.29,-10.23,0.92,204.48,1.73,50.42,0.53,1.542668e+10,1.486036e+10,2.687534,0.000232,26.653633,0,0,12.500,1.54,904.0,0.620000,4.255814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,-4.10,3.75,-6.26,3.94,52.53,3.88,20.08,1.02,5.546171e+09,1.594917e+10,4.689029,0.000031,29.507920,0,0,10.650,0.62,269.0,2.440000,-0.574367
1177,12.72,-3.83,-63.85,-4.55,-122.59,-10.98,117.47,0.41,-7.296394e+09,-1.397714e+10,2.357541,0.004177,20.000407,0,0,14.960,6.59,779.0,0.210000,0.000000
4957,-7.83,21.75,20.71,-11.89,-38.39,-108.37,8.14,0.11,-2.555557e+09,-4.958079e+09,29.717373,0.000064,2.964376,0,0,17.870,-178.00,0.0,-0.897169,-5.930419
999,0.04,-5.18,-18.77,-1.64,-40.13,-2.55,26.72,0.64,-2.795032e+09,1.114896e+10,3.393714,0.000297,28.585600,0,0,75.240,0.81,62.0,1.290000,-0.578231


In [ ]:
new_train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7994 entries, 0 to 7993
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   총자산증가율        7994 non-null   float64
 1   자기자본증가율       7994 non-null   float64
 2   매출액증가율        7994 non-null   float64
 3   총자본영업이익율      7994 non-null   float64
 4   자본금순이익율       7994 non-null   float64
 5   매출액영업이익율      7994 non-null   float64
 6   부채비율          7994 non-null   float64
 7   총자본회전율        7994 non-null   float64
 8   당기순이익         7994 non-null   float64
 9   영업활동으로인한현금흐름  7994 non-null   float64
 10  Z_score       7994 non-null   float64
 11  부도확률_O_score  7994 non-null   float64
 12  K_score       7994 non-null   float64
 13  자본잠식          7994 non-null   float64
 14  투자주의환기종목      7994 non-null   float64
 15  PER           7994 non-null   float64
 16  PBR           7994 non-null   float64
 17  EPS           7994 non-null   float64
 18  DIV           7994 non-null 

In [ ]:
new_train_X

,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,자본금순이익율,매출액영업이익율,부채비율,총자본회전율,당기순이익,영업활동으로인한현금흐름,Z_score,부도확률_O_score,K_score,자본잠식,투자주의환기종목,PER,PBR,EPS,DIV,EPS성장률
0,51.080000,-33.460000,-48.060000,-15.800000,-220.250000,-2804.730000,144.750000,0.010000,-1.479115e+10,-6.040461e+09,2.423710,0.004008,-6.172493,0,0,33.855000,7.060000,-1099.000000,0.000000,-3.303983
1,19.680000,5.230000,6.360000,5.300000,29.900000,3.400000,55.560000,1.560000,2.129262e+09,6.450040e+08,4.158521,0.000314,27.000318,0,0,25.600000,1.600000,142.000000,2.340000,-0.759729
2,69.710000,129.130000,1.760000,3.560000,-292.000000,3.730000,23.560000,0.960000,-1.180134e+10,2.859824e+09,10.550479,0.010310,11.189354,0,0,173.750000,1.070000,12.000000,0.000000,11.000000
3,-4.660000,-5.130000,-11.620000,-8.250000,-34.910000,-16.570000,15.730000,0.500000,-2.363712e+09,1.197800e+07,4.384619,0.000395,17.898521,1,0,41.360000,0.690000,-175.000000,0.000000,-0.356618
4,11.580000,12.290000,-10.230000,0.920000,204.480000,1.730000,50.420000,0.530000,1.542668e+10,1.486036e+10,2.687534,0.000232,26.653633,0,0,12.500000,1.540000,904.000000,0.620000,4.255814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7989,2.875680,-0.140566,-0.096901,13.928372,6.174862,-0.215061,-0.276170,-0.000223,-3.881622e-06,2.105156e+01,-0.000005,-0.203649,-0.016036,1,1,1.901807,-0.251282,14.245158,6.325268,-0.002757
7990,3.039021,4.749137,0.474862,-0.020844,8.278277,0.605429,-0.278447,5.364179,-8.761682e-03,1.568731e+01,12.269921,0.204560,0.835945,0,0,0.436274,-0.238882,3.706438,0.415492,-0.142993
7991,-0.164673,4.781502,2.923972,-0.000177,-0.195587,-0.278239,-0.213758,-0.001662,2.259668e-01,2.067266e+01,-0.000690,-0.008796,-0.025809,0,0,0.505224,-0.263866,4.659105,-0.154521,-0.023395
7992,0.199571,-0.235315,4.413585,-0.168828,-0.006359,-0.249359,-0.266267,0.020280,2.901691e+00,-3.971122e-11,15.265517,0.976613,8.870611,1,1,-0.182251,-0.203711,-0.003284,-0.054355,4.287870


In [ ]:
X_test

,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,자본금순이익율,매출액영업이익율,부채비율,총자본회전율,당기순이익,영업활동으로인한현금흐름,Z_score,부도확률_O_score,K_score,자본잠식,투자주의환기종목,PER,PBR,EPS,DIV,EPS성장률
1201,216.15,398.51,55.17,5.24,76.23,6.18,87.23,0.85,1.486632e+09,1.852755e+09,4.956898,0.000644,20.534538,0,0,49.780000,22.960000,678.000000,0.00,-0.260324
2994,7.60,2.60,34.21,1.91,65.01,3.10,51.02,0.62,5.003086e+09,1.420972e+10,2.849332,0.000365,27.662989,0,0,11.710000,1.290000,995.000000,0.86,-0.318960
3770,21.05,11.48,46.88,9.92,69.74,13.61,24.33,0.73,3.350148e+09,3.153563e+09,6.534388,0.000036,23.379505,0,0,39.110000,2.070000,123.000000,1.04,-0.875506
3904,88.21,168.42,57.92,12.12,16.60,6.82,48.73,1.78,7.069730e+08,1.032542e+10,8.110050,0.000310,22.535113,0,0,49.320000,15.940000,590.000000,0.00,0.645090
2192,54.15,67.39,4.91,0.52,271.07,1.29,50.11,0.40,4.197881e+10,2.528416e+09,3.271430,0.000023,19.502338,0,0,228.890000,3.600000,45.000000,0.00,-0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,13.73,25.10,31.27,12.42,117.77,11.72,81.38,1.06,3.276124e+09,4.741207e+09,0.000000,0.000337,20.233008,0,0,21.290000,5.220000,902.000000,0.00,0.549708
127,-19.78,-71.79,-23.62,-5.20,-60.20,-3.68,3185.53,1.41,-3.941386e+09,-4.407077e+09,0.000000,0.061125,13.113087,0,0,165.396667,44.413333,94.666667,0.00,0.128510
4737,-0.21,-0.76,-11.59,0.37,-128.26,0.53,95.55,0.69,-1.875594e+10,5.173831e+09,1.205787,0.002515,24.629430,0,0,3.160000,0.410000,923.000000,0.51,0.842315
1134,-2.94,-22.22,-15.08,-14.85,-162.73,-13.45,101.85,1.10,-7.870656e+09,-4.240453e+09,3.839628,0.003201,23.590757,0,0,74.640000,3.140000,211.000000,0.19,-0.854483


# 보영이 코드


In [ ]:
def gan_train(gan, generator, discriminator, catsh1,catsh2,catsh3,catsh4,catsh5,catsh6,catsh7,catsh8,catsh9,catsh10,catsh11,catsh12, catsh13, catsh14, catsh15, catsh16, catsh17, catsh18, catsh19, catsh20,catsh21, latent_dim, n_epochs, n_batch,n_eval):  # ,numerical
#, catsh22,catsh23, catsh24, catsh25, catsh26, catsh27,catsh28 , 
    half_batch = int(n_batch / 2)  
    discriminator_loss = []  
    generator_loss = []  
    valid = np.ones((half_batch, 1))  
    y_gan = np.ones((n_batch,1))   #, 1  
    # noise = np.random.normal(0, 1, size=[batch_size, random_dim])
    # y_gen = np.ones(batch_size)
    fake = np.zeros((half_batch, 1))  
    n_epochs = 7000 # 수정하렴! 

    for i in range(n_epochs):  
      idx = np.random.randint(0, catsh1.shape[0], half_batch)      

      총자산증가율_real = catsh1[idx]  
      자기자본증가율_real = catsh2[idx]  
      매출액증가율_real = catsh3[idx]  
      총자본영업이익율_real = catsh4[idx]  
      자본금순이익율_real = catsh5[idx]  
      매출액영업이익율_real = catsh6[idx]  
      부채비율_real = catsh7[idx]  
      총자본회전율_real = catsh8[idx]  
      당기순이익_real = catsh9[idx]  
      영업활동으로인한현금흐름_real = catsh10[idx]  
      Z_score_real = catsh11[idx]  
      부도확률_O_score = catsh12[idx]  
      K_score_real = catsh13[idx]  
      자본잠식_real = catsh14[idx]  
      투자주의환기종목_real = catsh15[idx]  
      날짜_real = catsh16[idx]  
      PER_real = catsh17[idx]  
      PBR_real = catsh18[idx]  
      EPS_real = catsh19[idx]  
      DIV_real = catsh20[idx]  
      EPS성장률_real = catsh21[idx]   
      # 벌금_real = catsh22[idx]  
      # 소송_real = catsh23[idx]    
      # 영업정지_real = catsh24[idx]          
      # 특허_real = catsh25[idx]          
      # 자본잠식_real = catsh26[idx]          
      # 투자주의환기종목_real = catsh27[idx]          
      # 업종_real = catsh28[idx]          
      # numerical_real = numerical[idx]

      real_data = np.concatenate([총자산증가율_real, 자기자본증가율_real, 매출액증가율_real ,총자본영업이익율_real,자본금순이익율_real,
                                  매출액영업이익율_real,부채비율_real ,총자본회전율_real, 당기순이익_real,영업활동으로인한현금흐름_real,
                                  Z_score_real,부도확률_O_score , K_score_real,자본잠식_real,  투자주의환기종목_real,
                                  날짜_real , PER_real , PBR_real, EPS_real, DIV_real,
                                  EPS성장률_real
                                  # ,소송_real , 영업정지_real, 특허_real,자본잠식_real, 투자주의환기종목_real,업종_real
                                  # ,numerical_real
                                  ], axis = 1)  


      noise = np.random.normal(0, 1, (half_batch, latent_dim))  
      fake_data = generator.predict(noise)  
    
      d_loss_real =  da_real = discriminator.train_on_batch(real_data, valid)  # 요기 수수께끼 
      d_loss_fake = da_fake = discriminator.train_on_batch(fake_data, fake)   # 요기 수수께끼 
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)   # 요기 수수께끼 
      discriminator_loss.append(d_loss)    # 요기 수수께끼  
      
      noise = np.random.normal(0, 1, (n_batch, latent_dim))  
      g_loss = gan.train_on_batch(noise, y_gan)  
      generator_loss.append(g_loss)  

      #evaluate progress
      if (i+1) % n_eval == 0:  
          print ("Epoch: %d [Discriminator loss: %f] [Generator loss: %f]" % (i + 1, d_loss, g_loss ))  
            
    plt.figure(figsize = (20, 10))  
    plt.plot(generator_loss, label = "Generator loss")  
    plt.plot(discriminator_loss, label = "Discriminator loss")     
    plt.title("Stats from training GAN")  
    plt.grid()  
    plt.legend()  

In [ ]:
latent_dim = 5000
gan_train(completegan, generator, discriminator,
          총자산증가율_numerical.values, 자기자본증가율_numerical.values, 매출액증가율_numerical.values , 총자본영업이익율_numerical.values,자본금순이익율_numerical.values,
          매출액영업이익율_numerical.values,부채비율_numerical.values,총자본회전율_numerical.values,당기순이익_numerical.values,영업활동으로인한현금흐름_numerical.values, 
          Z_score_numerical.values,부도확률_O_score_numerical.values,K_score_numerical.values,PER_numerical.values,PBR_numerical.values, 
          EPS_numerical.values,DIV_numerical.values,EPS성장률_numerical.values , 자본잠식_categorical.values , 투자주의환기종목_categorical.values , 날짜_categorical.values,
          latent_dim,
          n_epochs = 7000,
          n_batch = 63
          , n_eval = 200
          )

Epoch: 200 [Discriminator loss: 3.848173] [Generator loss: 0.007541]
Epoch: 400 [Discriminator loss: 4.182207] [Generator loss: 0.002163]
Epoch: 600 [Discriminator loss: 4.654871] [Generator loss: 0.001467]
Epoch: 800 [Discriminator loss: 5.449345] [Generator loss: 0.000914]
Epoch: 1000 [Discriminator loss: 5.632018] [Generator loss: 0.001010]


KeyboardInterrupt: ignored